# OCR(imgText) 전처리/임베딩/모델링 

In [ ]:
%pip install python-dotenv pymysql

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 환경 변수 로드
load_dotenv()

# 환경 변수에서 DB 암호 가져오기 (여기서는 변수명이 '1P_DB_PW'라고 가정)
db_password = os.getenv('1P_DB_PW')
print(db_password)

yttest1234


In [9]:
import pymysql
import pandas as pd
import ast
import re

# DB 접속 설정
db_config = {
    'host': '121.128.172.79',
    'user': 'user3',
    'password': db_password,  # db_password 변수에 비밀번호를 지정하세요.
    'database': 'yttest_db',
    'port': 3306,
    'cursorclass': pymysql.cursors.DictCursor
}

# DB 연결 및 데이터 불러오기
conn = pymysql.connect(**db_config)
cursor = conn.cursor()
# video 테이블에서 thumbnailURL, imgText, likeCount, viewCount, commentCount, catagoryID 컬럼을 가져옴
query = """SELECT v.thumbnailURL, t.imgText, v.likeCount, v.viewCount, v.commentCount, v.categoryID, v.videoID
            FROM video v
            JOIN thumbnail t ON v.thumbnailURL = t.thumbnailURL;"""
cursor.execute(query)
data = cursor.fetchall()
conn.close()  # 데이터 불러온 후 연결 종료

# DataFrame 생성
df = pd.DataFrame(data)

# 결측치 처리: imgText 컬럼의 결측값을 'unknown'으로 채워 넣기
df['imgText'] = df['imgText'].fillna('unknown')

# thumbnailURL 결측치 확인
if df['thumbnailURL'].isnull().sum() > 0:
    print("thumbnailURL 열에 결측치가 존재합니다. 추가 전처리가 필요합니다.")
else:
    print("thumbnailURL 열에 결측치가 없습니다.")

def clean_text(text):
    if pd.isnull(text):
        return ""
    # 1. 소문자화 (영어의 경우)
    text = text.lower()
    # 2. 공백 기준으로 단순 토큰화
    tokens = text.split()
    # 3. 각 토큰이 영어 알파벳이나 한글(가-힣)이 포함되어 있는 경우에만 유지
    cleaned_tokens = [token for token in tokens if re.search('[a-zA-Z가-힣]', token)]
    # 4. 다시 하나의 문자열로 결합
    return " ".join(cleaned_tokens)

# 예시: DB에서 불러온 imgText 열에 적용
df['cleaned_Texts'] = df['imgText'].apply(clean_text)

# 결과 확인 (예시)
print(df[['imgText', 'cleaned_Texts']].head())

# 저장 경로 (pkl 파일로 저장)
pkl_output_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_Texts_cleaned.pkl"

# DataFrame을 pickle 형식으로 저장 (모든 컬럼 포함: thumbnailURL, imgKw, cleaned_keywords, likeCount, viewCount, commentCount)
df.to_pickle(pkl_output_path)

print("전처리된 데이터가 .pkl 파일로 저장되었습니다.")

thumbnailURL 열에 결측치가 없습니다.
                                             imgText  \
0  개막전 티켓팅 오픈일정! 놓치면 후회! <광주 - NC: KIA SUP프로야구 프로...   
1                      *젊은 중년 여성들을 위한 데이트 팁과 노하우 5가지   
2  방콕에서는 스마트폰 가격이 한국의 2배라고./? [제가 쓰는 기종] 한국에서 이십만...   
3                43-3333 19 86689142121201 PRISM LIV   
4                               생각보다 취업사진 메이크업 준비해봤어   

                                       cleaned_Texts  
0  개막전 티켓팅 오픈일정! 놓치면 후회! <광주 nc: kia sup프로야구 프로야구매니저  
1                      *젊은 중년 여성들을 위한 데이트 팁과 노하우 5가지  
2  방콕에서는 스마트폰 가격이 한국의 2배라고./? [제가 쓰는 기종] 한국에서 이십만...  
3                                          prism liv  
4                               생각보다 취업사진 메이크업 준비해봤어  
전처리된 데이터가 .pkl 파일로 저장되었습니다.


In [10]:
# 결과 확인 (예시)
print(df[['imgText', 'cleaned_Texts']])

                                                  imgText  \
0       개막전 티켓팅 오픈일정! 놓치면 후회! <광주 - NC: KIA SUP프로야구 프로...   
1                           *젊은 중년 여성들을 위한 데이트 팁과 노하우 5가지   
2       방콕에서는 스마트폰 가격이 한국의 2배라고./? [제가 쓰는 기종] 한국에서 이십만...   
3                     43-3333 19 86689142121201 PRISM LIV   
4                                    생각보다 취업사진 메이크업 준비해봤어   
...                                                   ...   
101391      은환 은환 14 1333 #1..2151... 14 피해당 1037 Boatvar   
101392                                        누룽지 쿠키를 베이징   
101393  14회 용진중학 문회 한 인생 2막 14회 웅진중학교 10월 26일(토) 문회 한국...   
101394                                                      
101395   PATH EXILE 헌트리스 공개! [POE2 0.2.0 업데이트 신규직업 및 일정공개   

                                            cleaned_Texts  
0       개막전 티켓팅 오픈일정! 놓치면 후회! <광주 nc: kia sup프로야구 프로야구매니저  
1                           *젊은 중년 여성들을 위한 데이트 팁과 노하우 5가지  
2       방콕에서는 스마트폰 가격이 한국의 2배라고./? [제가 쓰는 기종] 한국에서 이십만...  
3                          

## kobert모델로 임베딩하면서 토큰화 동시 진행

In [17]:
data = pd.read_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_Texts_cleaned.pkl") 

    
print(data.info()) #["cleaned_keywords"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101396 entries, 0 to 101395
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   thumbnailURL   101396 non-null  object
 1   imgText        101396 non-null  object
 2   likeCount      101396 non-null  int64 
 3   viewCount      101396 non-null  int64 
 4   commentCount   101396 non-null  int64 
 5   categoryID     101396 non-null  object
 6   videoID        101396 non-null  object
 7   cleaned_Texts  101396 non-null  object
dtypes: int64(3), object(5)
memory usage: 6.2+ MB
None


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [11]:
from transformers import BertTokenizer, BertModel

c:\Users\YH\anaconda3\envs\model\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch
# --- 설정 및 장치 초기화 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [14]:
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model = model.to(device)
model.eval()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [15]:
# 간단한 텍스트 임베딩 함수 예시
def get_kobert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # GPU 사용 시
    with torch.no_grad():
        outputs = model(**inputs)
    # 토큰별 임베딩 평균 계산 (CLS 벡터 대안)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# 테스트 예시
embedding = get_kobert_embedding("안녕하세요, KoBERT 임베딩 테스트입니다.")
print(embedding)

[ 1.65561289e-02  3.96819785e-03  4.44694817e-01 -4.27673149e+00
 -7.64376447e-02  1.99296147e-01  2.70001404e-03  3.31027448e-01
 -2.86472082e-01 -5.93399525e-01 -1.16990879e-01  4.29399647e-02
 -8.63586962e-02  4.04513068e-02  4.89023745e-01 -2.79680230e-02
 -1.08981160e-02  3.53054330e-03  2.11349308e-01 -2.42854998e-01
 -2.62561440e-01 -1.14874102e-01 -6.15593866e-02  1.40246125e-02
 -2.25170165e-01  1.39964819e-01 -2.79441029e-01 -1.31088316e-01
  2.22674906e-01  1.42757729e-01  9.76208225e-02 -2.12782502e-01
  1.74072489e-01  1.23602599e-01  4.03719544e-01 -1.50353402e-01
  1.40943527e-01  4.88008708e-01  4.20410842e-01 -8.48478079e-01
  1.23377107e-01  2.69640516e-02  4.01190519e-01  2.65242513e-02
  4.63136017e-01  1.30413231e-02 -4.42528427e-01 -2.03680582e-02
  1.56477317e-01  1.25926033e-01  1.43534288e-01  1.56214848e-01
  1.06884152e-01  8.27481002e-02  2.04770640e-01 -8.76335800e-02
  4.65147406e-01 -4.38945979e-01 -2.86783397e-01  3.45532179e-01
 -2.24024639e-01 -1.87723

In [18]:
import ast

# 전처리 함수: 문자열 리스트 형식의 imgText를 파이썬 리스트로 변환한 뒤, 하나의 문장으로 결합
def process_text(text):
    try:
        tokens = ast.literal_eval(text)  # 문자열을 리스트로 변환
        if isinstance(tokens, list):
            return " ".join(tokens)       # 리스트를 공백으로 결합하여 하나의 문자열로 만듦
        else:
            return text
    except Exception as e:
        # 변환 실패 시 원본 텍스트 반환
        return text

# 이미 MariaDB에서 로딩한 df를 활용
# imgText 컬럼 전처리: 각 행의 텍스트를 하나의 문장으로 변환
df['processed_text'] = df['cleaned_Texts'].apply(process_text)

# KoBERT 임베딩 함수(get_kobert_embedding)가 이미 정의되어 있다고 가정합니다.
# 각 processed_text에 대해 임베딩 벡터를 계산하여 새로운 컬럼에 저장합니다.
df['embedding'] = df['processed_text'].apply(get_kobert_embedding)

# 결과 확인
print(df[['videoID', 'categoryID', 'processed_text', 'embedding']].head())

       videoID categoryID                                     processed_text  \
0  --0-PnOg3yk          5  개막전 티켓팅 오픈일정! 놓치면 후회! <광주 nc: kia sup프로야구 프로야구매니저   
1  --0Ql-Wv0TE          9                      *젊은 중년 여성들을 위한 데이트 팁과 노하우 5가지   
2  --3kZm7wt3I          1  방콕에서는 스마트폰 가격이 한국의 2배라고./? [제가 쓰는 기종] 한국에서 이십만...   
3  --6q2FHApro          1                                          prism liv   
4  --7YLVepuUg          6                               생각보다 취업사진 메이크업 준비해봤어   

                                           embedding  
0  [-0.050111175, 0.028868414, 0.021977657, -5.90...  
1  [0.3413147, -0.09835341, 0.09406995, -5.211521...  
2  [0.017420983, -0.10062581, 0.14835708, -6.1547...  
3  [0.21224183, 0.05399522, 0.120664574, -4.95183...  
4  [0.30539715, 0.05542153, 0.07300938, -6.116643...  


In [19]:
df.to_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_Texts_embedding.pkl")

In [20]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101396 entries, 0 to 101395
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   thumbnailURL    101396 non-null  object
 1   imgText         101396 non-null  object
 2   likeCount       101396 non-null  int64 
 3   viewCount       101396 non-null  int64 
 4   commentCount    101396 non-null  int64 
 5   categoryID      101396 non-null  object
 6   videoID         101396 non-null  object
 7   cleaned_Texts   101396 non-null  object
 8   processed_text  101396 non-null  object
 9   embedding       101396 non-null  object
dtypes: int64(3), object(7)
memory usage: 7.7+ MB
None


### 데이터셋 형성

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import json

# 이미 MariaDB에서 로딩한 df가 다음 컬럼들을 포함한다고 가정합니다.
# df: 'videoID', 'categoryID', 'viewCount', 'likeCount', 'commentCount', 'embedding', ...

# 1. 카테고리 전처리: One-Hot Encoding
encoder = OneHotEncoder(sparse_output=False)
# categoryID가 문자열 또는 숫자 형태라면 reshape 후 인코딩
category_encoded = encoder.fit_transform(df[['categoryID']])
print("One-Hot Encoded Category Shape:", category_encoded.shape)

# 2. OCR 임베딩 벡터 준비: 각 행의 embedding 컬럼을 2차원 배열로 변환
# embedding 컬럼은 이미 numpy 배열(예: shape=(embedding_dim,))를 포함하고 있어야 함
embedding_matrix = np.vstack(df['embedding'].values)
print("Embedding Matrix Shape:", embedding_matrix.shape)

# 3. 입력 피처 구성: embedding과 One-Hot encoded category 결합
# 두 배열을 column-wise로 결합합니다.
X = np.concatenate([embedding_matrix, category_encoded], axis=1)
print("Final Input Feature Shape:", X.shape)

# 4. 출력(Target) 변수 구성
# 출력 변수로 viewCount, likeCount, commentCount 사용
y = df[['viewCount', 'likeCount', 'commentCount']].values

# 선택 사항: 출력 값 분포가 매우 skewed하다면 로그 변환을 고려
y_log = np.log1p(y)  # log(1 + y) 변환

# 5. 학습 및 테스트 데이터 분리 (예: 80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

One-Hot Encoded Category Shape: (101396, 10)
Embedding Matrix Shape: (101396, 768)
Final Input Feature Shape: (101396, 778)
X_train shape: (81116, 778)
y_train shape: (81116, 3)


### 모델 학습

In [25]:
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import pickle

# 1. 모델 정의: XGBoost 회귀 모델을 MultiOutputRegressor로 감쌈
popularity_model = MultiOutputRegressor(
    xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
)

# 2. 모델 학습
popularity_model.fit(X_train, y_train)
print("모델 학습 완료!")

# 3. 예측 수행
y_pred = popularity_model.predict(X_test)

# 4. 평가 지표 계산
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("평가 결과:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R² Score:", r2)

# (선택 사항) 5. 모델 저장
df.to_pickle(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\thumbnail_Texts_model.pkl")
print("모델 저장 완료!")

모델 학습 완료!
평가 결과:
MAE: 1.9423021374359075
RMSE: 2.459083354472891
R² Score: 0.06305076808292605
모델 저장 완료!


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# --- 추천 함수 (앞서 정의한 recommend_thumbnail_text와 recommend_keywords_for_category 사용) ---
def recommend_thumbnail_text(candidate_text, category):
    """
    candidate_text: 후보 썸네일 문구 (문자열)
    category: 해당 문구의 카테고리 (DB와 일치하는 형식, 예: "2")
    
    반환: (추천 점수, 예측 인기도 벡터)
    """
    # 1. KoBERT 임베딩: candidate_text를 임베딩 벡터로 변환
    candidate_embedding = get_kobert_embedding(candidate_text)  # (임베딩 차원,)
    
    # 2. 카테고리 One-Hot 인코딩: 입력 값은 문자열 형태로 제공 (예: "2")
    category_encoded = encoder.transform([[str(category)]])
    
    # 3. 입력 피처 구성: candidate_embedding과 category_encoded 결합
    candidate_embedding = candidate_embedding.reshape(1, -1)
    candidate_feature = np.concatenate([candidate_embedding, category_encoded], axis=1)
    
    # 4. 인기도 예측: 회귀 모델(popularity_model)을 통해 후보 문구의 인기도 예측
    predicted_popularity = popularity_model.predict(candidate_feature)  # shape: (1, 3)
    
    # 5. 추천 점수 산출: 각 인기도 지표에 가중치를 부여하여 하나의 점수로 계산
    weights = np.array([0.5, 0.3, 0.2])
    score = (predicted_popularity * weights).sum()
    return score, predicted_popularity

def recommend_keywords_for_category(category, candidate_texts, top_n=5):
    """
    category: 추천할 카테고리 (DB 형식, 예: "2")
    candidate_texts: 해당 카테고리에 속하는 후보 썸네일 문구 리스트
    top_n: 상위 추천 문구 개수
    """
    recommendations = []
    for text in candidate_texts:
        score, _ = recommend_thumbnail_text(text, str(category))
        recommendations.append((text, score))
    # 추천 점수를 기준으로 내림차순 정렬
    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# 이미 정의한 추천 함수들을 사용해서 각 카테고리별 추천 결과를 모읍니다.
recommendations_dict = {}

unique_categories = df['categoryID'].unique()  # 예: [1, 2, 3, ...] 또는 문자열 형태

for cat in unique_categories:
    # 만약 categoryID가 숫자형이라면, 추천 함수에서는 문자열로 변환하여 사용합니다.
    candidate_texts = df[df['categoryID'] == cat]['processed_text'].tolist()
    recommendations = recommend_keywords_for_category(cat, candidate_texts, top_n=5)
    
    # recommendations는 각 추천 결과가 (문구, 추천 점수) 튜플로 되어 있음.
    # 이를 딕셔너리 형태로 저장: key: 카테고리, value: 추천 결과 리스트
    recommendations_dict[str(cat)] = [
        {"text": text, "score": score} for text, score in recommendations
    ]

import json

# recommendations_dict를 JSON 파일로 저장 (UTF-8로 한글 깨짐 방지)
with open(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\recommendations.json", "w", encoding="utf-8") as f:
    json.dump(recommendations_dict, f, ensure_ascii=False, indent=4)

print("추천 결과가 recommendations.json 파일에 저장되었습니다.")


추천 결과가 recommendations.json 파일에 저장되었습니다.


In [32]:
import json

# JSON 파일 열기 및 내용 읽기
with open(r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\recommendations.json", "r", encoding="utf-8") as f:
    loaded_recommendations = json.load(f)

# 모든 카테고리별 추천 문구 출력
for category, recs in loaded_recommendations.items():
    print(f"\n카테고리 {category} 추천 문구:")
    for idx, rec in enumerate(recs, 1):
        print(f"{idx}. {rec['text']} (추천 점수: {rec['score']:.4f})")


카테고리 5 추천 문구:
1. spotime 펄펄 날아다닌 '손흥민' 8분 만에 '2도움' 작렬 (추천 점수: 10.6409)
2. spotime 맨시티 '천적' 토트넘 직전 맞대결 '8분 요약' (추천 점수: 10.0834)
3. spotime 뮌헨, 3년 만의 승리 '천적' 레버쿠젠 꺾었다 laimer (추천 점수: 10.0135)
4. 런닝맨 pd들 pd들은 모두 고학력? gym종국 sbs pd 보다 짐종국 pd? 「양세찬은 지예은을 정말로 사랑해? 2025년 예능 pd의 자격 ironstein (추천 점수: 9.8616)
5. 이승훈 7년 만의 금메달! 스피드스케이팅 월드컵 5차 매스스타트 브스 스포츠 (추천 점수: 9.8518)

카테고리 9 추천 문구:
1. 비농업 취업자수/암호화폐 서밋 미국 증시 라이브 2025년 03월 07일 (금) (추천 점수: 8.2259)
2. “npc만 사는 도시 같다" 요즘 평양 모습 리뷰 (추천 점수: 8.0179)
3. 포항자유매국 김영주 단 국민이 식 투 지켜보고 김보 대한민국 1호프로 반공러 shpich (추천 점수: 7.9953)
4. ism 서비스업 pmi/베이지북 미국 증시 라이브 2025년 03월 05일 (수) (추천 점수: 7.8623)
5. 기준금리 결정/fomc 기자회견 미국 증시 라이브 2025년 03월 19일 (수) (추천 점수: 7.8270)

카테고리 1 추천 문구:
1. ohl 지디가 살아있는지 확인하는 매니저 매니저분이 와서 계속! |속!! (추천 점수: 10.3810)
2. 고 말씀을 야하나...? 고 말씀을 야하나...? 고경표를 알아본 행인들 반응 (추천 점수: 10.2924)
3. 데뷔각; 거를 타선이 없는 거를 데뷔가 확정된 순간 없는 데뷔 순간 (추천 점수: 10.0439)
4. co cl..???? what.??? (추천 점수: 10.0439)
5. 현실고증; 아이돌이 혈육을 아이 만나면 생기는 일 없을 만나고 일 (추천 점수: 10.0123)

카테고리 6 추천 문구:
1.

## LLM을 사용하여 후처리

In [33]:
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

In [34]:
import os
os.environ["OPENAI_API_KEY"]

In [ ]:
%pip install -U langchain langchain-openai

In [35]:
import os

def project_setup(name):
    os.environ['LANGSMITH_PROJECT'] = name
    os.environ['LANGSMITH_TRACING'] = 'true'
    print('프로젝트: {} 추적 시작'.format(name))

def stop_tracing():
    os.environ['LANGSMITH_TRACING'] = 'false'
    print('LangSmith 추적 중지')

project_setup('LangChain Test')

프로젝트: LangChain Test 추적 시작


In [36]:
OPENAI_MODELS = {
    "gpt-4o": {
        'price': {
            'input': 2.50, # per 1M tokens
            'cached': 1.25,
            'output': 10.00,
        }
    },
    "gpt-4o-mini": { # 가장 저렴한 모델
        'price': {
            'input': 0.15,
            'cached': 0.075,
            'output': 0.60,
        }
    },
    "o1": {
        'price': {
            'input': 15.00,
            'cached': 7.50,
            'output': 60.00,
        }
    },
    "o1-mini": {
        'price': {
            'input': 1.10,
            'cached': 0.55,
            'output': 4.40,
        }
    },
    # Usage Tier를 올려야 사용 가능
    "o3-mini": { # 가장 최신 모델 25-01-31
        'price': {
            'input': 1.10,
            'cached': 0.55,
            'output': 4.40,
        }
    },
}

MODEL_NAMES = list(OPENAI_MODELS.keys())

In [37]:
MODEL_NAMES

['gpt-4o', 'gpt-4o-mini', 'o1', 'o1-mini', 'o3-mini']

In [38]:
from langchain_openai import ChatOpenAI

model_name = "gpt-4o-mini"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048, # 최대 토큰수
    model_name=model_name,  # 모델명
) #.bind(logprobs=True)

In [42]:
import json

refined_recommendations = {}

for category, candidate_list in loaded_recommendations.items():
    refined_recommendations[category] = []

    if not candidate_list:
        continue

    for rec in candidate_list:
        candidate_text = rec["text"]

        prompt = f"""아래의 유튜브 썸네일 문구 후보를 더욱 자연스럽고 매력적인 문구로 다듬어 주세요.

후보 문구: "{candidate_text}"

최종 문구는 한국어로 작성해 주세요.
"""
        # content 속성으로 실제 문자열 결과 획득
        refined_caption = llm.invoke(prompt).content

        refined_recommendations[category].append({
            "original_text": candidate_text,
            "refined_text": refined_caption
        })

output_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\refined_recommendations.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(refined_recommendations, f, ensure_ascii=False, indent=4)

print("변환된 추천 문구가 refined_recommendations.json에 저장되었습니다.")

변환된 추천 문구가 refined_recommendations.json에 저장되었습니다.


In [43]:
import json

# 저장된 JSON 파일 경로
file_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\csv_model\refined_recommendations.json"

# JSON 파일을 불러오기
with open(file_path, "r", encoding="utf-8") as f:
    refined_data = json.load(f)

# 모든 카테고리별 추천 문구를 출력
for category, recommendations in refined_data.items():
    print(f"\n{'='*10} 카테고리 {category}의 추천 문구 {'='*10}\n")
    for idx, item in enumerate(recommendations, 1):
        original_text = item["original_text"]
        refined_text = item["refined_text"]
        
        print(f"{idx}. 원본: {original_text}")
        print(f"   변환: {refined_text}\n")


========== 카테고리 5의 추천 문구 ==========

1. 원본: spotime 펄펄 날아다닌 '손흥민' 8분 만에 '2도움' 작렬
   변환: "손흥민, Spotime에서 8분 만에 2도움 폭발!"

2. 원본: spotime 맨시티 '천적' 토트넘 직전 맞대결 '8분 요약'
   변환: "맨시티 vs 토트넘, '천적' 대결의 8분 하이라이트!"

3. 원본: spotime 뮌헨, 3년 만의 승리 '천적' 레버쿠젠 꺾었다 laimer
   변환: "3년 만에 승리! '천적' 레버쿠젠을 꺾은 뮌헨의 영웅, 라이머!"

4. 원본: 런닝맨 pd들 pd들은 모두 고학력? gym종국 sbs pd 보다 짐종국 pd? 「양세찬은 지예은을 정말로 사랑해? 2025년 예능 pd의 자격 ironstein
   변환: 최종 문구: "런닝맨 PD들은 모두 고학력? 짐종국 PD와 SBS PD의 대결! 양세찬은 지예은을 진짜 사랑할까? 2025년 예능 PD의 자격을 알아보자!"

5. 원본: 이승훈 7년 만의 금메달! 스피드스케이팅 월드컵 5차 매스스타트 브스 스포츠
   변환: "이승훈, 7년 만에 금메달! 스피드스케이팅 월드컵 5차 매스스타트 하이라이트"


========== 카테고리 9의 추천 문구 ==========

1. 원본: 비농업 취업자수/암호화폐 서밋 미국 증시 라이브 2025년 03월 07일 (금)
   변환: "2025년 3월 7일 (금) 비농업 취업자 수와 암호화폐 서밋, 미국 증시 라이브로 만나보세요!"

2. 원본: “npc만 사는 도시 같다" 요즘 평양 모습 리뷰
   변환: "‘NPC만 사는 도시?’ 요즘 평양의 모습 살펴보기"

3. 원본: 포항자유매국 김영주 단 국민이 식 투 지켜보고 김보 대한민국 1호프로 반공러 shpich
   변환: "포항 자유매국 김영주, 국민의 식탁을 지키는 김보! 대한민국 1호 프로 반공러의 이야기"

4. 원본: ism 서비스업 pmi/베이지북 미국 증시 라이브 2025년 03월 05일 (수)
   변환